In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# ベースURL
base_url = "https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13103&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

def scrape_all_pages():
    properties = []
    page = 1  # 初期ページ番号

    while True:
        url = base_url.format(page)
        print(f"[INFO] アクセス中: {url} (ページ {page})")
        try:
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"[ERROR] ステータスコード: {response.status_code}")
                break
        except Exception as e:
            print(f"[ERROR] リクエスト失敗: {e}")
            break

        # HTMLを解析
        soup = BeautifulSoup(response.text, "html.parser")

        # 各物件情報の取得
        property_items = soup.find_all("div", class_="ui-media-body property_unit-info")
        if not property_items:
            print("[INFO] 物件情報が見つかりません。このページが最後です。")
            break

        for item in property_items:
            try:
                # 物件名
                name_tag = item.find("dd", class_="dottable-vm")
                name = name_tag.text.strip() if name_tag else None

                # 販売価格
                price_tag = item.find("span", class_="dottable-value")
                price = price_tag.text.strip() if price_tag else None

                # 所在地
                address_tag = item.find(lambda tag: tag.name == "dt" and "所在地" in tag.text)
                address = (
                    address_tag.find_next_sibling("dd").text.strip() if address_tag else None
                )

                # 専有面積と間取り
                detail_table = item.find("table", class_="dottable-fix")
                area = None
                layout = None
                if detail_table:
                    area_tag = detail_table.find(lambda tag: tag.name == "dt" and "専有面積" in tag.text)
                    layout_tag = detail_table.find(lambda tag: tag.name == "dt" and "間取り" in tag.text)
                    area = area_tag.find_next("dd").text.strip() if area_tag else None
                    layout = layout_tag.find_next("dd").text.strip() if layout_tag else None

                # データをリストに追加
                properties.append({
                    "物件名": name,
                    "販売価格": price,
                    "所在地": address,
                    "専有面積": area,
                    "間取り": layout,
                })

            except AttributeError as e:
                print(f"[WARNING] データ取得失敗: {e}")
                continue

        # 次のページ
        page += 1
        time.sleep(3)  # サーバー負荷軽減のため待機

    return properties

# スクレイピング開始
print("[INFO] スクレイピングを開始します...")
properties = scrape_all_pages()

# データをCSVに保存
if properties:
    df = pd.DataFrame(properties)
    output_file = "minato_properties_no_built_year.csv"
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"[INFO] データをCSVに保存しました: {output_file}")
    print(f"[INFO] 総物件数: {len(properties)} 件")
else:
    print("[INFO] データが取得できませんでした。")


[INFO] スクレイピングを開始します...
[INFO] アクセス中: https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13103&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999 (ページ 1)
[INFO] アクセス中: https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13103&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999 (ページ 2)
[INFO] アクセス中: https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13103&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999 (ページ 3)
[INFO] アクセス中: https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13103&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999 (ページ 4)
[INFO] アクセス中: https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13103&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999 (ページ 5)
[INFO] アクセス